
# Preprocess Microsoft Academics Graph (MAG) Dataset

Jupyter Notebook for the preprocessing of the Microsoft Academics Graph (MAG) dump.

For this process, the following CSV files are needed: ```ConferenceInstances.txt```, ```ConferenceSeries.txt```, ```Papers.txt```. 
The above files can be found here: https://archive.org/download/mag-2021-06-07/mag/

In particular, the following operations are going to be executed:
* Opening of ConferenceInstances and ConferenceSeries CSVs
* Drop of the useless columns 
* Chuncked Processing of the Papers CSV
    * Drop of the useless columns
    * Drop of papers without DOI
    * Drop of papers from journals and books rows
* Merge with the processed conferences data
* Fix of some missing conferences locations with queries to the DBLP website

Lastly, the entire preprocessed dump is going to be saved on disk in CSV format

In [1]:
# Libraries Import
import pandas as pd
import platform
import multiprocessing as mp 
import concurrent       
from preprocess_multithread_utils import * 

pd.set_option('display.max_columns', None)

## File Paths
Please set your working directory paths.

In [4]:
# ******************* PATHS ********************+

# Dumps Directory Path
path_file_import = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Import/'

# CSV Exports Directory Path
path_file_export = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/'

### Use a Previuously Preprocessed Papers CSV
This can be really useful to save some time using a previously elaborated CSV file. We don't need to repeat the same operations!

**Note**: the CSV needs to be in the same format of the one generated with this script

In [3]:
# Use a Previuously Preprocessed Papers CSV
#
# This can be really useful to save some time using a previously elaborated
# CSV file. We don't need to repeat the same operations!
#
# Note: the CSV needs to be in the same format of the one generated with this script
read_preprocessed_papers = True
preprocessed_papers_csv_path = r'/Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_mag_papers.csv'

### Multithreading Settings
Settings needed for the multithreaded queries to gather the missing conferences locations from the DBLP website.

Please specify the number of CPU threads below:

In [5]:
n_cpu_threads = 8 # Number of CPU threads

Special setting for the specific operating systems.

**Note**: Due to the latest MacOS releases' security measures, we need to use the spawn method instead of fork.

In [6]:
print(f"Notebook running on {platform.system()} OS: ")

if platform.system() == "Darwin" or platform.system() == "Windows": # MacOS and windows
    mp_ctx = mp.get_context("spawn")
    print("Spawn method has been set")
    
else: # other unix systems
    mp_ctx = mp.get_context("fork")
    print("Spawn method has been set")

Notebook running on Darwin OS: 
Spawn method has been set


## Preprocess of Conference Instances CSV

In [12]:
# ******************* CONFERENCE INSTANCES ********************

# Read of the Conference Instances File

# The column names follow the MAG' scheme official documentation
df_mag_conf_instances_col_names = ['ConferenceInstanceID', 'NormalizedName', 'DisplayName', 'ConferenceSeriesID', 'Location', 'OfficialUrl', 'StartDate', 'EndDate', 'AbstractRegistrationDate', 'SubmissionDeadlineDate', 'NotificationDueDate', 'FinalVersionDueDate', 'PageCount', 'PaperFamilyCount', 'CitationCount', 'Latitude', 'Longitude', 'CreatedDate']

df_mag_conf_instances = pd.read_csv(path_file_import + 'ConferenceInstances.txt', sep='\t', names=df_mag_conf_instances_col_names)
df_mag_conf_instances

,ConferenceInstanceID,NormalizedName,DisplayName,ConferenceSeriesID,Location,OfficialUrl,StartDate,EndDate,AbstractRegistrationDate,SubmissionDeadlineDate,NotificationDueDate,FinalVersionDueDate,PageCount,PaperFamilyCount,CitationCount,Latitude,Longitude,CreatedDate
0,7785157,time 2008,TIME 2008,2624631009,"Montreal, Canada",http://www.time2008.org/,2008-06-16,2008-06-18,NaN,2008-01-11,NaN,NaN,23,23,319,45.512400,-73.554680,2016-06-24
1,15420687,ipmu 2008,IPMU 2008,1128239323,"Malaga, Spain",http://www.gimac.uma.es/ipmu08,2008-06-22,2008-06-27,NaN,2007-12-07,NaN,NaN,5,5,45,36.718320,-4.420160,2016-06-24
2,16798864,wosn 2010,WOSN 2010,2756885533,"Boston, MA, USA",http://www.usenix.org/events/wosn10/cfp/,2010-06-22,2010-06-22,NaN,2010-02-25,2010-04-30,2010-05-25,9,9,666,42.358660,-71.056740,2016-06-24
3,18230910,sasn 2009,SASN 2009,1128894334,Saint Petersburg (Russia),http://www.ieee-sasn.org/index.html,2009-10-12,2009-10-14,2009-06-19,2009-06-26,2009-07-31,2009-09-11,0,0,0,59.933180,30.306030,2016-06-24
4,31227610,eurocon 2011,EUROCON 2011,1190350587,"Lisbon, Portugal",http://www.eurocon2011.it.pt/,2011-04-27,2011-04-29,NaN,2010-10-30,2011-01-30,2011-02-28,279,279,864,38.725700,-9.150250,2016-06-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16454,2890348533,icn 2019,ICN 2019,1147859159,"Valencia, Spain",http://iaria.org/conferences2019/ICN19.html,2019-03-24,2019-03-28,NaN,2018-11-10,2019-01-10,2019-02-04,179,179,180,39.468990,-0.376860,2018-09-27
16455,2890856803,ismco'2019,ISMCO'2019,2898428697,"Incline Village,us",http://www.ismco.net,2019-04-29,2019-05-01,NaN,2018-12-17,NaN,NaN,6,6,12,39.250090,-119.959267,2018-09-27
16456,2891744759,ro2018,RO2018,2898131225,"Amsterdam,nl",http://www.researchobject.org/ro2018/,2018-10-29,2018-10-29,2018-07-15,2018-07-15,NaN,NaN,1,1,0,52.353218,5.002769,2018-09-27
16457,2892113557,icccn 2019,ICCCN 2019,1137850760,"Valencia, Spain",http://icccn.org/icccn19,2019-07-29,2019-08-01,NaN,2019-03-01,2019-04-26,2019-05-10,126,126,224,39.468990,-0.376860,2018-09-27


Here the useless columns are going to be removed from the dataframe.

In [13]:
# Drop of Conference Instances' Useless Columns
df_mag_conf_instances = df_mag_conf_instances.drop(columns=['OfficialUrl', 'AbstractRegistrationDate', 'SubmissionDeadlineDate', 'NotificationDueDate', 'FinalVersionDueDate', 'PageCount', 'PaperFamilyCount', 'CitationCount', 'Latitude', 'Longitude', 'CreatedDate', 'StartDate', 'EndDate'])
df_mag_conf_instances

,ConferenceInstanceID,NormalizedName,DisplayName,ConferenceSeriesID,Location
0,7785157,time 2008,TIME 2008,2624631009,"Montreal, Canada"
1,15420687,ipmu 2008,IPMU 2008,1128239323,"Malaga, Spain"
2,16798864,wosn 2010,WOSN 2010,2756885533,"Boston, MA, USA"
3,18230910,sasn 2009,SASN 2009,1128894334,Saint Petersburg (Russia)
4,31227610,eurocon 2011,EUROCON 2011,1190350587,"Lisbon, Portugal"
...,...,...,...,...,...
16454,2890348533,icn 2019,ICN 2019,1147859159,"Valencia, Spain"
16455,2890856803,ismco'2019,ISMCO'2019,2898428697,"Incline Village,us"
16456,2891744759,ro2018,RO2018,2898131225,"Amsterdam,nl"
16457,2892113557,icccn 2019,ICCCN 2019,1137850760,"Valencia, Spain"


Column rename to remove ambiguity for the future joins

In [14]:
# Column rename to remove ambiguity for the future joins
df_mag_conf_instances.rename(columns={'NormalizedName': 'ConferenceNormalizedName', 'DisplayName': 'ConferenceDisplayName', 'Location': 'ConferenceLocation'}, inplace=True)
df_mag_conf_instances

,ConferenceInstanceID,ConferenceNormalizedName,ConferenceDisplayName,ConferenceSeriesID,ConferenceLocation
0,7785157,time 2008,TIME 2008,2624631009,"Montreal, Canada"
1,15420687,ipmu 2008,IPMU 2008,1128239323,"Malaga, Spain"
2,16798864,wosn 2010,WOSN 2010,2756885533,"Boston, MA, USA"
3,18230910,sasn 2009,SASN 2009,1128894334,Saint Petersburg (Russia)
4,31227610,eurocon 2011,EUROCON 2011,1190350587,"Lisbon, Portugal"
...,...,...,...,...,...
16454,2890348533,icn 2019,ICN 2019,1147859159,"Valencia, Spain"
16455,2890856803,ismco'2019,ISMCO'2019,2898428697,"Incline Village,us"
16456,2891744759,ro2018,RO2018,2898131225,"Amsterdam,nl"
16457,2892113557,icccn 2019,ICCCN 2019,1137850760,"Valencia, Spain"


## Preprocess of Conference Series CSV

In [15]:
# ******************* CONFERENCE SERIES ********************

# Read of the Conference Series File

# The column names follow the MAG' scheme official documentation
df_mag_conf_series_col_names = ['ConferenceSeriesID', 'Rank', 'NormalizedName', 'DisplayName', 'PaperCount', 'PaperFamilyCount', 'CitationCount', 'CreatedDate']

df_mag_conf_series = pd.read_csv(path_file_import + 'ConferenceSeries.txt', sep='\t', names=df_mag_conf_series_col_names)
df_mag_conf_series

,ConferenceSeriesID,Rank,NormalizedName,DisplayName,PaperCount,PaperFamilyCount,CitationCount,CreatedDate
0,1134804816,12817,ICIDS,International Conference on Interactive Digita...,611,610,2945,2016-06-24
1,1165160117,14777,SWAT4LS,Semantic Web Applications and Tools for Life S...,85,85,213,2016-06-24
2,1192093291,12271,TRIDENTCOM,Testbeds and Research Infrastructures for the ...,571,571,5174,2016-06-24
3,1199066382,10155,BIOINFORMATICS,International Conference on Bioinformatics,10692,10692,17021,2016-06-24
4,1201746639,15567,AIS,Autonomous and Intelligent Systems,165,165,1002,2016-06-24
...,...,...,...,...,...,...,...,...
4533,2754809603,14461,IPSS,IEEE International Power Sources Symposium,101,101,188,2017-09-25
4534,2756271167,13527,ECMS,European Conference on Modelling and Simulation,283,283,915,2017-09-25
4535,2756896743,17566,CAI,Conference on Algebraic Informatics,124,124,567,2017-10-06
4536,2757378734,15053,UPGRADE-CN,"Use of P2P, GRID and Agents for the Developmen...",40,40,314,2017-10-06


Here the useless columns are going to be removed from the dataframe.

In [16]:
# Drop of Conference Series' Useless Columns
df_mag_conf_series = df_mag_conf_series.drop(columns=['Rank', 'PaperCount', 'PaperFamilyCount', 'CitationCount', 'CreatedDate'])
df_mag_conf_series

,ConferenceSeriesID,NormalizedName,DisplayName
0,1134804816,ICIDS,International Conference on Interactive Digita...
1,1165160117,SWAT4LS,Semantic Web Applications and Tools for Life S...
2,1192093291,TRIDENTCOM,Testbeds and Research Infrastructures for the ...
3,1199066382,BIOINFORMATICS,International Conference on Bioinformatics
4,1201746639,AIS,Autonomous and Intelligent Systems
...,...,...,...
4533,2754809603,IPSS,IEEE International Power Sources Symposium
4534,2756271167,ECMS,European Conference on Modelling and Simulation
4535,2756896743,CAI,Conference on Algebraic Informatics
4536,2757378734,UPGRADE-CN,"Use of P2P, GRID and Agents for the Developmen..."


Column rename to remove ambiguity for the future joins

In [17]:
# Column rename to remove ambiguity for the future joins
df_mag_conf_series.rename(columns={'NormalizedName': 'ConferenceSeriesNormalizedName', 'DisplayName': 'ConferenceSeriesDisplayName'}, inplace=True)
df_mag_conf_series

,ConferenceSeriesID,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName
0,1134804816,ICIDS,International Conference on Interactive Digita...
1,1165160117,SWAT4LS,Semantic Web Applications and Tools for Life S...
2,1192093291,TRIDENTCOM,Testbeds and Research Infrastructures for the ...
3,1199066382,BIOINFORMATICS,International Conference on Bioinformatics
4,1201746639,AIS,Autonomous and Intelligent Systems
...,...,...,...
4533,2754809603,IPSS,IEEE International Power Sources Symposium
4534,2756271167,ECMS,European Conference on Modelling and Simulation
4535,2756896743,CAI,Conference on Algebraic Informatics
4536,2757378734,UPGRADE-CN,"Use of P2P, GRID and Agents for the Developmen..."


## Preprocess of Papers CSV
The Papers CSV is going to be processed in chunks, due to its size.

The following operations are going to be executed:
* Drop of the useless columns
* Filtering of papers without DOI
* Filtering papers that are not related to conferences
* Drop of the doctype column
* Write of the processed file on disk (in CSV format)

In [18]:
# ******************* PAPERS ********************

# Read of previously prerocessed CSV
df_mag_papers = None
if read_preprocessed_papers:
    df_mag_papers = pd.read_csv(preprocessed_papers_csv_path, low_memory=False, index_col=0)
else:
    # The Papers CSV is going to be processed in chunks, due to its size

    # The column names follow the MAG' scheme official documentation
    df_mag_papers_col_names = ['PaperID', 'Rank', 'Doi', 'DocType', 'PaperTitle', 'OriginalTitle', 'BookTitle', 'Year', 'Date', 'OnlineDate', 'Publisher', 'JournalID', 'ConferenceSeriesID', 'ConferenceInstanceID', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'CitationCount', 'EstimatedCitation', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'Retracion', 'CreatedDate']

    # List of processed chunks.
    df_mag_papers_list_of_chunks = list()

    # Define of the chunk size
    chunksize = 10 ** 7

    count = 1
    with pd.read_csv(path_file_import + 'Papers.txt', sep='\t', chunksize=chunksize, low_memory=False, on_bad_lines='skip', names=df_mag_papers_col_names) as reader:
        for chunk in reader:

            # Drop of the useless columns
            chunk = chunk.drop(columns=['Rank', 'OnlineDate', 'Publisher', 'Volume', 'Issue', 'FirstPage', 'LastPage', 'ReferenceCount', 'OriginalVenue', 'FamilyID', 'FamilyRank', 'Retracion', 'CreatedDate', 'JournalID', 'BookTitle', 'Date'])

            # Filtering of papers without DOI
            chunk = chunk.dropna(subset = ['Doi'])

            # Filtering papers that are not related to conferences
            chunk = chunk[chunk.DocType == 'Conference']

            # Drop of the doctype column
            chunk = chunk.drop(columns=['DocType'])

            # Insert of the resulting chunk in the list 
            df_mag_papers_list_of_chunks.append(chunk)

            print(f'Successfully processed chunk {count} out of around {260000000 / chunksize}')
            count += 1
            break

    # Concatenation of the processed chunks
    df_mag_papers = pd.concat(df_mag_papers_list_of_chunks)

    # Empty the list to free some memory
    df_mag_papers_list_of_chunks = list()

    # Write of the resulting CSV on Disk
    df_mag_papers.to_csv(path_file_export + 'out_mag_papers.csv')
    print(f'Successfully Exported the Preprocessed Papers CSV to {path_file_export}out_mag_papers.csv')

df_mag_papers

,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation
37,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014.0,1.131603e+09,4038532.0,12.0,12
39,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014.0,1.154039e+09,157008481.0,10.0,10
68,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013.0,1.196984e+09,NaN,20.0,20
197,60437532,10.1007/3-540-46146-9_77,similarity image retrieval system using hierar...,Similarity Image Retrieval System Using Hierar...,2002.0,1.192665e+09,NaN,0.0,0
666,198056957,10.1007/11785231_94,leukemia prediction from gene expression data ...,Leukemia prediction from gene expression data—...,2006.0,1.176896e+09,NaN,19.0,19
...,...,...,...,...,...,...,...,...,...
259718386,3102242761,10.1109/IECON43393.2020.9254316,loss reduction by synchronous rectification in...,Loss Reduction by Synchronous Rectification in...,2020.0,2.623572e+09,NaN,0.0,0
259718500,3136855299,10.1109/BMSB49480.2020.9379806,data over cable services improving the bicm ca...,Data Over Cable Services – Improving the BICM ...,2020.0,2.623662e+09,NaN,0.0,0
259718537,3145351916,10.1109/ACC.1988.4172843,model reference robust adaptive control withou...,Model Reference Robust Adaptive Control withou...,1988.0,2.238538e+09,NaN,0.0,0
259718570,3151696876,10.1109/ICASSP.2002.1005676,missing data speech recognition in reverberant...,Missing data speech recognition in reverberant...,2002.0,1.121228e+09,NaN,0.0,0


## Merge of Conferences and Papers Data

In [19]:
# ******************* MERGE OF CONFERENCES AND PAPERS DATA ********************

# Merge of conferences and papers data over the conferenceseries id columnn to get the conference series name
# The papers' row that will not match will be preserved
df_mag_preprocessed = pd.merge(df_mag_papers, df_mag_conf_series, on=['ConferenceSeriesID'], how='left')

# Merge of conferences and papers data over the conferenceinstances id columnn
# The papers' row that will not match will be preserved
df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_mag_conf_instances, on=['ConferenceInstanceID'], how='left')

# Drop of the duplicated columns
df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceSeriesID_y'])
df_mag_preprocessed.rename(columns = {'ConferenceSeriesID_x':'ConferenceSeriesID'}, inplace=True)

# Removing broken data (four records seems to have mismatched types in some columns)
df_mag_preprocessed = df_mag_preprocessed.dropna(subset = ['CitationCount'])

df_mag_preprocessed

,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName,ConferenceNormalizedName,ConferenceDisplayName,ConferenceLocation
0,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014.0,1.131603e+09,4038532.0,12.0,12,DISC,International Symposium on Distributed Computing,disc 2014,DISC 2014,"Austin, TX"
1,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014.0,1.154039e+09,157008481.0,10.0,10,ESA,European Symposium on Algorithms,esa 2014,ESA 2014,"Wrocław, Poland"
2,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013.0,1.196984e+09,NaN,20.0,20,ENTER,Information and Communication Technologies in ...,NaN,NaN,NaN
3,60437532,10.1007/3-540-46146-9_77,similarity image retrieval system using hierar...,Similarity Image Retrieval System Using Hierar...,2002.0,1.192665e+09,NaN,0.0,0,DEXA,Database and Expert Systems Applications,NaN,NaN,NaN
4,198056957,10.1007/11785231_94,leukemia prediction from gene expression data ...,Leukemia prediction from gene expression data—...,2006.0,1.176896e+09,NaN,19.0,19,ICAISC,International Conference on Artificial Intelli...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409811,3102242761,10.1109/IECON43393.2020.9254316,loss reduction by synchronous rectification in...,Loss Reduction by Synchronous Rectification in...,2020.0,2.623572e+09,NaN,0.0,0,IECON,Conference of the Industrial Electronics Society,NaN,NaN,NaN
4409812,3136855299,10.1109/BMSB49480.2020.9379806,data over cable services improving the bicm ca...,Data Over Cable Services – Improving the BICM ...,2020.0,2.623662e+09,NaN,0.0,0,BMSB,International Symposium on Broadband Multimedi...,NaN,NaN,NaN
4409813,3145351916,10.1109/ACC.1988.4172843,model reference robust adaptive control withou...,Model Reference Robust Adaptive Control withou...,1988.0,2.238538e+09,NaN,0.0,0,ACC,American Control Conference,NaN,NaN,NaN
4409814,3151696876,10.1109/ICASSP.2002.1005676,missing data speech recognition in reverberant...,Missing data speech recognition in reverberant...,2002.0,1.121228e+09,NaN,0.0,0,ICASSP,"International Conference on Acoustics, Speech,...",NaN,NaN,NaN


## Fix of the Mismatched Data Type

Fix of the year data type, that has been interpreted as a float.

In [20]:
type(df_mag_preprocessed.iloc[:1]["Year"][0])

numpy.float64

In [21]:
df_mag_preprocessed = df_mag_preprocessed.astype({"Year": int}, errors='raise') 
df_mag_preprocessed.iloc[:3]

,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName,ConferenceNormalizedName,ConferenceDisplayName,ConferenceLocation
0,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014,1.131603e+09,4038532.0,12.0,12,DISC,International Symposium on Distributed Computing,disc 2014,DISC 2014,"Austin, TX"
1,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014,1.154039e+09,157008481.0,10.0,10,ESA,European Symposium on Algorithms,esa 2014,ESA 2014,"Wrocław, Poland"
2,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013,1.196984e+09,NaN,20.0,20,ENTER,Information and Communication Technologies in ...,NaN,NaN,NaN


Fix of the CitationCount data type, that has been interpreted as a float.

In [22]:
type(df_mag_preprocessed.iloc[:1]["CitationCount"][0])

numpy.float64

In [23]:
df_mag_preprocessed = df_mag_preprocessed.astype({"CitationCount": int}, errors='raise') 
df_mag_preprocessed.iloc[:3]

,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName,ConferenceNormalizedName,ConferenceDisplayName,ConferenceLocation
0,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014,1.131603e+09,4038532.0,12,12,DISC,International Symposium on Distributed Computing,disc 2014,DISC 2014,"Austin, TX"
1,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014,1.154039e+09,157008481.0,10,10,ESA,European Symposium on Algorithms,esa 2014,ESA 2014,"Wrocław, Poland"
2,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013,1.196984e+09,NaN,20,20,ENTER,Information and Communication Technologies in ...,NaN,NaN,NaN


## Fix of the Missing Conferences Locations
Some papers have only the indication of the conference series. For this reason, the conference instance and the related conference locations don't have a value.

However, every paper has been published in a specific "instance" of a conference, hence it should have a location. These papers will be "fixed" considering the year of their publication and their conference.

In [24]:
df_mag_preprocessed_subset = df_mag_preprocessed.iloc[:50]
df_mag_preprocessed_subset = df_mag_preprocessed_subset.dropna(subset = ['ConferenceNormalizedName'])
df_mag_preprocessed_subset.iloc[:10][["Year", "ConferenceSeriesNormalizedName", "ConferenceNormalizedName", "ConferenceDisplayName"]]

,Year,ConferenceSeriesNormalizedName,ConferenceNormalizedName,ConferenceDisplayName
0,2014,DISC,disc 2014,DISC 2014
1,2014,ESA,esa 2014,ESA 2014
7,2011,LTC,ltc 2011,LTC 2011
8,2013,CVPR,cvpr 2013,CVPR 2013
14,2008,BMSB,bmsb 2008,BMSB 2008
16,2000,CAV,cav 2000,CAV 2000
19,2008,ISVC,isvc 2008,ISVC 2008
25,2000,CLEO,cleo 2000,CLEO 2000
33,2014,ICC,icc 2014,ICC 2014
35,2000,CRYPTO,crypto 2000,CRYPTO 2000


As you can see in the above test, the ConferenceNormalizedName seems to be made by the concatenation of ConferenceSeriesNormalizedName in lowercase, a space, and the papers' year.

**Note**: in the above subset the ConferenceDisplayName seems to be composed in the same way of ConferenceNormalizedName, but without the lowercase. However, this is not always true!

Now we're going to populate the ConferenceNormalizedName instances that don't have a value.

In [25]:
df_mag_preprocessed.ConferenceNormalizedName.fillna(df_mag_preprocessed.ConferenceSeriesNormalizedName.str.lower() + ' ' + df_mag_preprocessed.Year.astype(str), inplace=True)
df_mag_preprocessed

,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName,ConferenceNormalizedName,ConferenceDisplayName,ConferenceLocation
0,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014,1.131603e+09,4038532.0,12,12,DISC,International Symposium on Distributed Computing,disc 2014,DISC 2014,"Austin, TX"
1,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014,1.154039e+09,157008481.0,10,10,ESA,European Symposium on Algorithms,esa 2014,ESA 2014,"Wrocław, Poland"
2,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013,1.196984e+09,NaN,20,20,ENTER,Information and Communication Technologies in ...,enter 2013,NaN,NaN
3,60437532,10.1007/3-540-46146-9_77,similarity image retrieval system using hierar...,Similarity Image Retrieval System Using Hierar...,2002,1.192665e+09,NaN,0,0,DEXA,Database and Expert Systems Applications,dexa 2002,NaN,NaN
4,198056957,10.1007/11785231_94,leukemia prediction from gene expression data ...,Leukemia prediction from gene expression data—...,2006,1.176896e+09,NaN,19,19,ICAISC,International Conference on Artificial Intelli...,icaisc 2006,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409811,3102242761,10.1109/IECON43393.2020.9254316,loss reduction by synchronous rectification in...,Loss Reduction by Synchronous Rectification in...,2020,2.623572e+09,NaN,0,0,IECON,Conference of the Industrial Electronics Society,iecon 2020,NaN,NaN
4409812,3136855299,10.1109/BMSB49480.2020.9379806,data over cable services improving the bicm ca...,Data Over Cable Services – Improving the BICM ...,2020,2.623662e+09,NaN,0,0,BMSB,International Symposium on Broadband Multimedi...,bmsb 2020,NaN,NaN
4409813,3145351916,10.1109/ACC.1988.4172843,model reference robust adaptive control withou...,Model Reference Robust Adaptive Control withou...,1988,2.238538e+09,NaN,0,0,ACC,American Control Conference,acc 1988,NaN,NaN
4409814,3151696876,10.1109/ICASSP.2002.1005676,missing data speech recognition in reverberant...,Missing data speech recognition in reverberant...,2002,1.121228e+09,NaN,0,0,ICASSP,"International Conference on Acoustics, Speech,...",icassp 2002,NaN,NaN


I tried to do a new merge with the Conference Instances dataframe (this time it will be made on the ConferenceNormalizedName column), but I had no luck: these conference instances are missing. That's probably the reason of the NaN values in the ConferenceInstanceID field of the original Papers table.

In [ ]:
df_mag_conf_instances.loc[df_mag_conf_instances["ConferenceNormalizedName"] == "enter 2013"]

### Gathering the Missing Conferences Locations from the DBLP Website
The missing conferences locations are going to be obtained from queries to the DBLP Website.

In [ ]:
df_mag_conferences = df_mag_preprocessed[["ConferenceNormalizedName", "ConferenceLocation"]]

Drop of the papers that don't need their location to be fixed.

In [8]:
df_mag_conferences = df_mag_conferences[df_mag_conferences["ConferenceLocation"].isna()]
df_mag_conferences

,ConferenceNormalizedName,ConferenceLocation
2,enter 2013,NaN
3,dexa 2002,NaN
4,icaisc 2006,NaN
5,interact 2011,NaN
6,fct 2005,NaN
...,...,...
4409807,iecon 2020,NaN
4409808,bmsb 2020,NaN
4409809,acc 1988,NaN
4409810,icassp 2002,NaN


Drop of the duplicated conferences. We only need unique values.

In [9]:
df_mag_conferences = df_mag_conferences.drop_duplicates(subset="ConferenceNormalizedName")

print(f"Now we only need to search for the location of {df_mag_conferences.__len__()} unique conferences")

Now we only need to search for the location of 29512 unique conferences


#### Define of the Web Scraping Function

In [28]:
def dblp_location_scraper(conferences_dataframe, dblp_url = "https://dblp.org/db/conf/"):
    dict_conf_locations = {}      
    download_list = list(conferences_dataframe.ConferenceNormalizedName.values)

    executor = concurrent.futures.ProcessPoolExecutor(max_workers=n_cpu_threads * 10, mp_context=mp_ctx)
    futures = [executor.submit(mt_get_mag_conf_location_from_dblp_operation, conf_name, dblp_url) for conf_name in download_list]

    for future in concurrent.futures.as_completed(futures):
        try:
            k, v = future.result()
        except Exception as e:
            print(f"{futures[future]} throws {e}")
        else:
            dict_conf_locations[k] = v
            pass

    # Converting the resulting dictionary to a dataframe
    df_conf_locations = pd.DataFrame(dict_conf_locations.items(), columns=['ConferenceNormalizedName', 'ConferenceLocation'])

    return df_conf_locations

#### Queries to https://dblp.org/db/conf/

Parallel execution of the queries to the DBLP website.

**Note**: this operation should take less than 10min, depending on your Internet speed.

In [ ]:
df_conf_locations = dblp_location_scraper(df_mag_conferences, "https://dblp.org/db/conf/")

In [10]:
dblp_url = "https://dblp.org/db/conf/"
dict_conf_locations = {}      
download_list = list(df_mag_conferences.ConferenceNormalizedName.values)

executor = concurrent.futures.ProcessPoolExecutor(max_workers=n_cpu_threads * 10, mp_context=mp_ctx)
futures = [executor.submit(mt_get_mag_conf_location_from_dblp_operation, conf_name, dblp_url) for conf_name in download_list]

for future in concurrent.futures.as_completed(futures):
    try:
        k, v = future.result()
    except Exception as e:
        print(f"{futures[future]} throws {e}")
    else:
        dict_conf_locations[k] = v
        pass

# Converting the resulting dictionary to a dataframe
df_conf_locations = pd.DataFrame(dict_conf_locations.items(), columns=['ConferenceNormalizedName', 'ConferenceLocation'])

Let's see how many conference locations have been fixed.

In [30]:
df_conf_locations = df_conf_locations.dropna(subset = ['ConferenceLocation'])

print(f"Fixed {len(df_conf_locations.index)} over {len(df_mag_conferences.index)} unique conferences")

Fixed 287 over 29512 unique conferences


#### Queries to https://dblp.org/db/series/

We're going to try to get more location composing the URL in a different way.

First of all, we have to filter the conferences that have already been obtained:

In [40]:
df_mag_conferences_v2 = df_mag_conferences.copy()
rows_to_drop = df_mag_conferences_v2["ConferenceNormalizedName"].isin(df_conf_locations["ConferenceNormalizedName"])
df_mag_conferences_v2.drop(df_mag_conferences_v2[rows_to_drop].index, inplace=True)

print(f"Now we only need to search for the location of {df_mag_conferences_v2.__len__()} unique conferences")

Now we only need to search for the location of 29225 unique conferences


**Note**: in my tests, this method gave no results. I decided to leave the original code, in case something will change on the DBLP website. You can execute the download anyway if you want, by editing the following value. 

In [47]:
download_anyway = False

In [44]:
if download_anyway:
    df_conf_locations_2 = dblp_location_scraper(df_mag_conferences_v2, "https://dblp.org/db/series/")

Let's see how many conference locations have been fixed.

In [45]:
if download_anyway:
    df_conf_locations_2 = df_conf_locations_2.dropna(subset = ['ConferenceLocation'])

    print(f"Fixed {len(df_conf_locations_2.index)} over {len(df_mag_conferences.index)} unique conferences")

Fixed 0 over 29512 unique conferences


#### Join of the New Location Data with the Original Dataframe

In [48]:
# Merge with the first location dataframe
df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_conf_locations, on=['ConferenceNormalizedName'], how='left')

# Combine the two columns
df_mag_preprocessed['ConferenceLocation_x'] = df_mag_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
df_mag_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceLocation_y'])

if download_anyway:
    # Merge with the second location dataframe
    df_mag_preprocessed = pd.merge(df_mag_preprocessed, df_conf_locations_2, on=['ConferenceNormalizedName'], how='left')

    # Combine the two columns
    df_mag_preprocessed['ConferenceLocation_x'] = df_mag_preprocessed['ConferenceLocation_x'].fillna(df_mag_preprocessed['ConferenceLocation_y'])
    df_mag_preprocessed.rename(columns = {'ConferenceLocation_x':'ConferenceLocation'}, inplace=True)
    df_mag_preprocessed = df_mag_preprocessed.drop(columns=['ConferenceLocation_y'])

df_mag_preprocessed

,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName,ConferenceNormalizedName,ConferenceDisplayName,ConferenceLocation
0,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014,1.131603e+09,4038532.0,12,12,DISC,International Symposium on Distributed Computing,disc 2014,DISC 2014,"Austin, TX"
1,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014,1.154039e+09,157008481.0,10,10,ESA,European Symposium on Algorithms,esa 2014,ESA 2014,"Wrocław, Poland"
2,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013,1.196984e+09,NaN,20,20,ENTER,Information and Communication Technologies in ...,enter 2013,NaN,"Innsbruck, Austria"
3,60437532,10.1007/3-540-46146-9_77,similarity image retrieval system using hierar...,Similarity Image Retrieval System Using Hierar...,2002,1.192665e+09,NaN,0,0,DEXA,Database and Expert Systems Applications,dexa 2002,NaN,"Aix-en-Provence, France"
4,198056957,10.1007/11785231_94,leukemia prediction from gene expression data ...,Leukemia prediction from gene expression data—...,2006,1.176896e+09,NaN,19,19,ICAISC,International Conference on Artificial Intelli...,icaisc 2006,NaN,"Zakopane, Poland"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409807,3102242761,10.1109/IECON43393.2020.9254316,loss reduction by synchronous rectification in...,Loss Reduction by Synchronous Rectification in...,2020,2.623572e+09,NaN,0,0,IECON,Conference of the Industrial Electronics Society,iecon 2020,NaN,NaN
4409808,3136855299,10.1109/BMSB49480.2020.9379806,data over cable services improving the bicm ca...,Data Over Cable Services – Improving the BICM ...,2020,2.623662e+09,NaN,0,0,BMSB,International Symposium on Broadband Multimedi...,bmsb 2020,NaN,NaN
4409809,3145351916,10.1109/ACC.1988.4172843,model reference robust adaptive control withou...,Model Reference Robust Adaptive Control withou...,1988,2.238538e+09,NaN,0,0,ACC,American Control Conference,acc 1988,NaN,NaN
4409810,3151696876,10.1109/ICASSP.2002.1005676,missing data speech recognition in reverberant...,Missing data speech recognition in reverberant...,2002,1.121228e+09,NaN,0,0,ICASSP,"International Conference on Acoustics, Speech,...",icassp 2002,NaN,NaN


Count of how many paper's conference locations are still missing

In [53]:
n_missing = len(df_mag_preprocessed.index) - len(df_mag_preprocessed.dropna(subset = ['ConferenceLocation']).index)
print(f"{n_missing} missing paper's conference locations")

2915559 missing paper's conference locations


## Write of the Final CSV on Disk

In [51]:
# Write of the resulting CSV on Disk
df_mag_preprocessed.to_csv(path_file_export + 'out_mag_citations_count_and_conferences.csv')
print(f'Successfully Exported the Preprocessed CSV to {path_file_export}out_mag_citations_count_and_conferences.csv')

Successfully Exported the Preprocessed CSV to /Users/marcoterzulli/File/Scuola Local/Magistrale/Materiale Corsi Attuali/Tirocinio/Cartella di Lavoro/Archivi Dump di Lavoro/Export/out_mag_citations_count_and_conferences.csv


Check of the Exported CSV to be sure that everything went fine.

In [52]:
# Check of the Exported CSV
df_mag_exported_csv = pd.read_csv(path_file_export + 'out_mag_citations_count_and_conferences.csv', low_memory=False)
df_mag_exported_csv

,Unnamed: 0,PaperID,Doi,PaperTitle,OriginalTitle,Year,ConferenceSeriesID,ConferenceInstanceID,CitationCount,EstimatedCitation,ConferenceSeriesNormalizedName,ConferenceSeriesDisplayName,ConferenceNormalizedName,ConferenceDisplayName,ConferenceLocation
0,0,14558443,10.1007/978-3-662-45174-8_28,the adaptive priority queue with elimination a...,The Adaptive Priority Queue with Elimination a...,2014,1.131603e+09,4038532.0,12,12,DISC,International Symposium on Distributed Computing,disc 2014,DISC 2014,"Austin, TX"
1,1,15354235,10.1007/978-3-662-44777-2_60,document retrieval on repetitive collections,Document Retrieval on Repetitive Collections,2014,1.154039e+09,157008481.0,10,10,ESA,European Symposium on Algorithms,esa 2014,ESA 2014,"Wrocław, Poland"
2,2,24327294,10.1007/978-3-319-03973-2_13,socomo marketing for travel and tourism,SoCoMo Marketing for Travel and Tourism,2013,1.196984e+09,NaN,20,20,ENTER,Information and Communication Technologies in ...,enter 2013,NaN,"Innsbruck, Austria"
3,3,60437532,10.1007/3-540-46146-9_77,similarity image retrieval system using hierar...,Similarity Image Retrieval System Using Hierar...,2002,1.192665e+09,NaN,0,0,DEXA,Database and Expert Systems Applications,dexa 2002,NaN,"Aix-en-Provence, France"
4,4,198056957,10.1007/11785231_94,leukemia prediction from gene expression data ...,Leukemia prediction from gene expression data—...,2006,1.176896e+09,NaN,19,19,ICAISC,International Conference on Artificial Intelli...,icaisc 2006,NaN,"Zakopane, Poland"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4409807,4409807,3102242761,10.1109/IECON43393.2020.9254316,loss reduction by synchronous rectification in...,Loss Reduction by Synchronous Rectification in...,2020,2.623572e+09,NaN,0,0,IECON,Conference of the Industrial Electronics Society,iecon 2020,NaN,NaN
4409808,4409808,3136855299,10.1109/BMSB49480.2020.9379806,data over cable services improving the bicm ca...,Data Over Cable Services – Improving the BICM ...,2020,2.623662e+09,NaN,0,0,BMSB,International Symposium on Broadband Multimedi...,bmsb 2020,NaN,NaN
4409809,4409809,3145351916,10.1109/ACC.1988.4172843,model reference robust adaptive control withou...,Model Reference Robust Adaptive Control withou...,1988,2.238538e+09,NaN,0,0,ACC,American Control Conference,acc 1988,NaN,NaN
4409810,4409810,3151696876,10.1109/ICASSP.2002.1005676,missing data speech recognition in reverberant...,Missing data speech recognition in reverberant...,2002,1.121228e+09,NaN,0,0,ICASSP,"International Conference on Acoustics, Speech,...",icassp 2002,NaN,NaN


Order by citations count descending to see the articles with the most citations

In [ ]:
# Order by citations count descending to see the articles with the most citations
df_mag_exported_csv = df_mag_exported_csv.sort_values(by='CitationCount', ascending=False)
df_mag_exported_csv